Dependencies

In [1]:
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    DataCollatorForLanguageModeling, 
    Trainer, 
    TrainingArguments
)

from datasets import load_dataset

In [2]:
dataset = load_dataset('hakurei/open-instruct-v1', split='train')
dataset.to_pandas().sample(20)

Found cached dataset json (C:/Users/cristian.sanchezp/.cache/huggingface/datasets/hakurei___json/hakurei--open-instruct-v1-00713eb9aefc6002/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


,output,instruction,input
198451,"SELECT customer_id, customer_name, customer_sa...","Write a SQL query to find the customer id, nam...",
303426,"example_animal = Animal(name=""Fido"", species=""...",Create an example instance of the given class.,Class: Animal
319200,Photosynthesis is a process in which plants ma...,Explain the process of photosynthesis in a sim...,
330086,Product,Find the top 10 most popular products in this ...,
457858,It's always a good idea to start by identifyin...,"Hello, do you have an idea for how i should st...",
321892,"Once upon a time, there was a lonely fruit bat...",Edit the following paragraph to improve senten...,"Once upon time there was a lonely fruit bat, J..."
452291,"Bonjour,\n\nJe suis ravi de vous aider en tant...",Can you please produce an output with the foll...,
68448,"""Plant""","Given a string, predict which category it belo...",
491374,Psychological treatments for depression are tr...,Tell me about psychological treatments for dep...,
137578,Strength training offers numerous benefits for...,Imagine you are a personal trainer advising a ...,


In [3]:
def preprocess(example):
    example['prompt'] = f"{example['instruction']} {example['input']} {example['output']}"
    return example

In [4]:
dataset = dataset.map(preprocess, remove_columns=['instruction','input','output'])

Loading cached processed dataset at C:\Users\cristian.sanchezp\.cache\huggingface\datasets\hakurei___json\hakurei--open-instruct-v1-00713eb9aefc6002\0.0.0\e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4\cache-cbda1d7d0dbdd8ca.arrow


In [5]:
dataset.to_pandas().sample(10)

,prompt
46534,Write an opening statement for a presentation ...
139015,You are given the dimensions and the weight of...
63101,Tell me what happened on <date> in history.\n\...
214874,How can I be healthy ? Being healthy is not a...
365087,How do you think about the future of technolog...
301132,"Based on the provided text, edit the mistakes...."
253056,When was West Ham started and when did they ge...
372138,Count how many vowels are there in a given str...
199846,Write a Java program that adds two numbers and...
370991,Can you write a program that can identify if t...


In [6]:
dataset = dataset.shuffle(seed=42).select(range(100000)).train_test_split(test_size=0.1)

Loading cached shuffled indices for dataset at C:\Users\cristian.sanchezp\.cache\huggingface\datasets\hakurei___json\hakurei--open-instruct-v1-00713eb9aefc6002\0.0.0\e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4\cache-2bc5d54f439219b6.arrow


In [7]:
train_dataset = dataset['train']
test_dataset = dataset['test']

Model Selection

In [8]:
MODEL_NAME = "microsoft/DialoGPT-medium"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [10]:
def tokenize_dataset(dataset):
    return dataset.map(lambda example: tokenizer(example['prompt'], truncation=True, max_length=128), batched=True, remove_columns=['prompt'])

In [11]:
tokenizer.pad_token = tokenizer.eos_token

In [12]:
train_dataset = tokenize_dataset(train_dataset)
test_dataset = tokenize_dataset(test_dataset)

Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [13]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

The trainer will use this to process the input and create appropiate batches for training, as we use the generative model, the mlm will be set to false

In [16]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [15]:
training_args = TrainingArguments(
    output_dir='../../data/interim/',
    num_train_epochs=1, #To keep things fast
    per_device_eval_batch_size=4,
    per_device_train_batch_size=8,
)

In [18]:
#Will do all the heavy lifting
trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_dataset, 
    eval_dataset=test_dataset, 
    data_collator=data_collator,
)

In [19]:
trainer.train()
trainer.save_model()

  0%|          | 0/5625 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: [enforce fail at C:\b\abs_abjetg6_iu\croot\pytorch_1686932924616\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 408488896 bytes.